# Проект по SQL - Анализ базы данных книжного мазагина

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Цель исследования** 

Проанализировать базу данных книжного магазина. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задания**
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Импорт библиотек и подключение к СУБД

In [1]:
# импорти библиотек
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': '###', # пароль
'host': '###',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Формирование SQL запроса

In [3]:
# Формируем sql-запрос - функция для выполнения запроса и возвращению результата
def sql_result(query):
    return pd.io.sql.read_sql(query, con = engine)

books = ''' SELECT * FROM books'''
authors = ''' SELECT * FROM authors'''
ratings = ''' SELECT * FROM ratings'''
reviews = ''' SELECT * FROM reviews'''
publishers = ''' SELECT * FROM publishers'''

In [4]:
# Выведу первые строки из каждого полученного датафрейма
for x in [books, authors, ratings, reviews, publishers]:
    display(sql_result(x).head(2))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...


,publisher_id,publisher
0,1,Ace
1,2,Ace Book


## Задания

### Задание 1

Посчитайте, сколько книг вышло после 1 января 2000 года

In [5]:
task_1 = ''' SELECT COUNT(book_id)
               FROM books
              WHERE DATE_TRUNC('day', publication_date) > '2000-01-01';
         '''
sql_result(task_1)

,count
0,819


**Вывод:**
После 1 января 2000 года вышло 819 книг.

---

### Задание 2

Для каждой книги посчитайте количество обзоров и среднюю оценку

Датафреймы: books, ratings, reviews

In [22]:
task_2 = '''WITH
            rew_c AS(
                SELECT b.book_id, b.title, COUNT(re.review_id) AS count_reviews
                FROM reviews as re
                FULL OUTER JOIN books AS b ON re.book_id = b.book_id
                GROUP BY b.book_id
            ),
            rat_a AS(
                SELECT b.book_id, AVG(ra.rating) as avg_rating
                FROM ratings as ra
                LEFT JOIN books AS b ON ra.book_id = b.book_id
                GROUP BY b.book_id
            )
            SELECT rew_c.book_id, rew_c.title, rew_c.count_reviews, rat_a.avg_rating
            FROM rew_c
            LEFT JOIN rat_a ON rew_c.book_id = rat_a.book_id; 
        '''
sql_result(task_2)

,book_id,title,count_reviews,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


In [15]:
task_2 = '''WITH
            r_c AS(
                SELECT b.book_id, b.title, COUNT(re.review_id) AS count_reviews
                FROM reviews as re
                FULL OUTER JOIN books AS b ON re.book_id = b.book_id
                GROUP BY b.book_id
            ),
            r_a AS(
                SELECT b.book_id, AVG(ra.rating) as avg_rating
                FROM ratings as ra
                LEFT JOIN books AS b ON ra.book_id = b.book_id
                GROUP BY b.book_id
            )
            SELECT r_c.book_id, r_c.title, r_c.count_reviews, r_a.avg_rating
            FROM r_c
            LEFT JOIN r_a ON r_c.book_id = r_a.book_id; 
        '''
sql_result(task_2)

,book_id,title,count_reviews,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


**Вывод:**

Для всех книг посчитано количество обзоров и средний рейтинг. У 6 книг отсутствуют обзоры, таким образом, они не попали в расчётную таблицу.

---


### Задание 3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

Датафреймы: books, publishers

In [7]:
task_3 = ''' SELECT p.publisher, COUNT(b.title)
             FROM books AS b
             JOIN publishers AS p ON b.publisher_id = p.publisher_id
             WHERE b.num_pages > 50
             GROUP BY p.publisher
             ORDER BY count DESC
             LIMIT 1;
         '''
sql_result(task_3)

,publisher,count
0,Penguin Books,42


**Вывод:**

Penguin Books выпустило самое большое число книг, толщиной более 50 страниц.

---


### Задание 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

Датафреймы: books, authors, ratings

In [8]:
task_4 = ''' SELECT a.author, AVG(r.rating) as rating_avg
             FROM books AS b
             JOIN ratings AS r ON b.book_id = r.book_id
             JOIN authors AS a ON b.author_id = a.author_id
             WHERE b.book_id IN (SELECT book_id
                                 FROM ratings
                                 GROUP BY book_id
                                 HAVING COUNT(rating_id) > 50)
             GROUP BY a.author
             ORDER BY rating_avg DESC
             LIMIT 1;
         '''
sql_result(task_4)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.287097


**Вывод:**

Среди авторов книг с более 50-ю оценками лидируют Джоан Роулинг и Мэри Гранд.

---


### Задание 5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

Датафреймы: books, ratings, reviews

In [9]:
task_5 = ''' WITH
            rat AS(
                    SELECT username, COUNT(rat.rating) as rating_count
                    FROM ratings as rat
                    GROUP BY username
                    HAVING COUNT(rat.rating) > 50
            ),
            rew AS(
                    SELECT username, COUNT(review_id) AS review_count
                    FROM reviews as rew
                    GROUP BY username
            )
            SELECT AVG(rew.review_count) AS user_reviews_avg
            FROM rat
            LEFT JOIN rew ON rat.username = rew.username;
         '''
sql_result(task_5)

,user_reviews_avg
0,24.333333


**Вывод:**

В среднем активные пользователи (те, кто поставил более 50 оценок) написали порядка 24 обзоров.